In [1]:
# Initialize required random seeds before importing anything else.
import random

import numpy as np
from tensorflow import set_random_seed

np.random.seed(42)
set_random_seed(42)
random.seed(42)

/home/qkt/.local/share/virtualenvs/sgan-techniques-WxACZbr4/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%matplotlib inline

import json

from keras.optimizers import RMSprop
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import sgantech as sgt

Using TensorFlow backend.


In [3]:
sns.set_style('whitegrid')
color_palette = sns.color_palette()

In [4]:
(x_train, y_train), (x_test, y_test) = sgt.utils.load_preprocessed_mnist()
X = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))

In [5]:
n_critic = 5
clip_value = 0.01

In [6]:
def train_test_gan(gan_factory, x_train, y_train, x_test, y_test, epochs=40000, batch_size=32, test_steps=500, num_labels=None, verbose=True):
    classes = np.unique(y)
    num_classes = len(classes)
    
    quarter_batch = batch_size // 4
    cw1 = {-1: 1, 1: 1}
    cw2 = {i: 10 / quarter_batch for i in range(num_classes)}
    cw2[num_classes] = 1 / quarter_batch
    
    if num_labels is None:
        x_train_labeled = x_train
        y_train_labeled = y_train
    else:
        x_train_labeled, _, y_train_labeled, _ = train_test_split(x_train, y_train, train_size=num_labels, stratify=y_train)
    
    disc, gen, combined = gan_factory()
    
    histories = []
    for epoch in range(epochs):
        for _ in range(n_critic):
            # Select a random quater batch of images for real/fake
            real_fake_idx = np.random.randint(0, x_train.shape[0], quarter_batch)
            real_fake_imgs = x_train[real_fake_idx]
            # Select a random quarter batch of labeled images
            labeled_idx = np.random.randint(0, x_train_labeled.shape[0], quarter_batch)
            labeled_imgs = x_train_labeled[labeled_idx]
            # Sample noise and generate a half batch of new images
            noise = np.random.normal(0, 1, (quarter_batch * 2, 100))
            gen_imgs = gen.predict(noise)

            valid = -np.ones((quarter_batch, 1))
            fake = np.ones((quarter_batch, 1))

            labels = to_categorical(y_train_labeled[labeled_idx], num_classes=num_classes+1)
            fake_labels = to_categorical(np.full((quarter_batch, 1), num_classes), num_classes=num_classes+1)

            # Train the discriminator
            d_loss_real = disc.train_on_batch([real_fake_imgs, labeled_imgs], [valid, labels], class_weight=[cw1, cw2])
            d_loss_fake = disc.train_on_batch([gen_imgs[:quarter_batch], gen_imgs[quarter_batch:]], [fake, fake_labels], class_weight=[cw1, cw2])
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            for l in disc.layers:
                weights = l.get_weights()
                weights = [np.clip(w, -clip_value, clip_value) for w in weights]
                l.set_weights(weights)

        noise = np.random.normal(0, 1, (batch_size, 100))
        validity = -np.ones((batch_size, 1))

        # Train the generator
        g_loss = combined.train_on_batch(noise, validity, class_weight=[cw1, cw2])
        
        if epoch % test_steps == 0:
            test_loss = disc.evaluate(
                [x_test, x_test],
                [np.ones((len(x_test), 1)), to_categorical(y_test, num_classes=num_classes+1)]
            )
            print(test_loss)
            histories.append((d_loss, g_loss, test_loss))

        # Plot the progress
        if verbose:
            print("%d [D loss: %f, acc: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss))
    
    test_loss = disc.evaluate(
        [x_test, x_test],
        [np.ones((len(x_test), 1)), to_categorical(y_test, num_classes=num_classes+1)]
    )
    histories.append(test_loss)
    
    return histories

In [7]:
def dense_wgan_factory():
    dense_d = sgt.discriminators.dense((28, 28, 1), 10, unsupervised_activation='linear')
    dense_g = sgt.generators.dense((100,), (28, 28, 1))
    return sgt.gan.SGANBuilder(dense_g, dense_d, (28, 28, 1), (100,)) \
        .set_discriminator_loss([sgt.losses.wasserstein_loss, 'categorical_crossentropy']) \
        .set_generator_loss(sgt.losses.wasserstein_loss) \
        .set_discriminator_optimizer(RMSprop(lr=0.00005)) \
        .set_generator_optimizer(RMSprop(lr=0.00005)) \
        .build()

In [8]:
dense_histories = train_test_gan(dense_wgan_factory, x_train, y_train, x_test, y_test, epochs=12500, verbose=False)

/home/qkt/.local/share/virtualenvs/sgan-techniques-WxACZbr4/lib/python3.5/site-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


10000/10000 [==============================] - 0s 46us/step
[1.197403161239624, -0.0019956754026934503, 2.3968020027160644, 0.0, 0.103]
10000/10000 [==============================] - 0s 49us/step
[14.05979585723877, 25.764546923828124, 2.35504483795166, 0.0, 0.1135]
10000/10000 [==============================] - 0s 49us/step
[13.275085575866699, 24.134525231933594, 2.415645907974243, 0.0, 0.1028]
10000/10000 [==============================] - 0s 49us/step
[12.388497633361816, 22.343237283325195, 2.433757963180542, 0.0, 0.098]
10000/10000 [==============================] - 0s 50us/step
[9.716625354003906, 17.01142036743164, 2.421830376815796, 0.0, 0.1009]
10000/10000 [==============================] - 0s 50us/step
[4.770108943176269, 7.135394436645508, 2.404823447418213, 0.0, 0.1009]
10000/10000 [==============================] - 0s 49us/step
[1.9379847164154054, 1.4741973217010498, 2.4017721138000487, 0.9228, 0.1135]
10000/10000 [==============================] - 0s 49us/step
[1.306562

In [9]:
def cnn_wgan_factory():
    cnn_d = sgt.discriminators.cnn((28, 28, 1), 10, unsupervised_activation='linear')
    cnn_g = sgt.generators.cnn((100,), (7, 7))
    return sgt.gan.SGANBuilder(cnn_g, cnn_d, (28, 28, 1), (100,)) \
        .set_discriminator_loss([sgt.losses.wasserstein_loss, 'categorical_crossentropy']) \
        .set_generator_loss(sgt.losses.wasserstein_loss) \
        .set_discriminator_optimizer(RMSprop(lr=0.00005)) \
        .set_generator_optimizer(RMSprop(lr=0.00005)) \
        .build()

In [10]:
cnn_histories = train_test_gan(cnn_wgan_factory, x_train, y_train, x_test, y_test, epochs=12500, verbose=False)

/home/qkt/.local/share/virtualenvs/sgan-techniques-WxACZbr4/lib/python3.5/site-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


10000/10000 [==============================] - 3s 309us/step
[1.1990313787460327, 0.0001704237805912271, 2.397892288208008, 0.0, 0.101]
10000/10000 [==============================] - 3s 290us/step
[1.1996344678878783, -0.008763673540949821, 2.4080326103210448, 0.0, 0.0]
10000/10000 [==============================] - 3s 290us/step
[1.2075552717208862, 0.0247183290630579, 2.3903922092437746, 0.0, 0.204]
10000/10000 [==============================] - 3s 290us/step
[1.1926410427093506, -0.0023240239698439835, 2.387606108093262, 0.0, 0.2056]
10000/10000 [==============================] - 3s 290us/step
[1.1855406366348267, -0.012687110218405724, 2.3837683769226072, 0.0, 0.1938]
10000/10000 [==============================] - 3s 290us/step
[1.1844341131210328, -0.015254679092764855, 2.3841229038238527, 0.0, 0.098]
10000/10000 [==============================] - 3s 291us/step
[1.1839132236480714, -0.014246545204520225, 2.382072996902466, 0.0, 0.1767]
10000/10000 [==============================] 

In [13]:
from sklearn.model_selection import train_test_split

dense_ss_histories = {}
for num_examples in [100, 300, 500, 1000]:
    print(num_examples)
    dense_ss_histories[num_examples] = train_test_gan(
        dense_wgan_factory, x_train, y_train, x_test, y_test, epochs=50000, num_labels=num_examples, verbose=False
    )

100


/home/qkt/.local/share/virtualenvs/sgan-techniques-WxACZbr4/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/qkt/.local/share/virtualenvs/sgan-techniques-WxACZbr4/lib/python3.5/site-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


10000/10000 [==============================] - 1s 64us/step
[1.204011713409424, 0.01119626025557518, 2.3968271759033204, 0.0, 0.1199]
10000/10000 [==============================] - 1s 51us/step
[14.620000622558594, 26.868520629882813, 2.3714805931091307, 0.0, 0.1032]
10000/10000 [==============================] - 1s 54us/step
[14.04086590270996, 25.670133981323243, 2.411597789764404, 0.0, 0.0958]
10000/10000 [==============================] - 1s 53us/step
[13.270549453735352, 24.146482904052736, 2.3946159759521484, 0.0, 0.1135]
10000/10000 [==============================] - 1s 54us/step
[11.830445603942872, 21.2529085357666, 2.407982676315308, 0.0, 0.1135]
10000/10000 [==============================] - 1s 52us/step
[6.83843620300293, 11.269701022338868, 2.4071713973999023, 0.0, 0.0982]
10000/10000 [==============================] - 1s 52us/step
[2.838068974685669, 3.272713101196289, 2.403424839019775, 0.0, 0.1032]
10000/10000 [==============================] - 1s 51us/step
[1.211424487

10000/10000 [==============================] - 1s 50us/step
[0.7828291113853455, 0.01385477720350027, 1.5518034448623657, 0.0, 0.5597]
10000/10000 [==============================] - 1s 50us/step
[0.7624292971611023, -0.02800359134078026, 1.5528621864318848, 0.0, 0.5728]
10000/10000 [==============================] - 1s 51us/step
[0.7772472512245179, -0.011806803105771542, 1.5663013063430786, 0.0, 0.5386]
10000/10000 [==============================] - 0s 50us/step
[0.7939010718345642, 0.015701901811361312, 1.5721002403259277, 0.0, 0.5378]
10000/10000 [==============================] - 1s 50us/step
[0.7279104077339172, -0.09065493549108505, 1.5464757480621338, 0.0, 0.5707]
10000/10000 [==============================] - 0s 46us/step
[0.7706083152770996, 0.0023804808925837278, 1.5388361476898194, 0.0, 0.5643]
10000/10000 [==============================] - 0s 50us/step
[0.7722240639686584, -0.010966439636424184, 1.5554145656585694, 0.0, 0.5399]
10000/10000 [==============================] -

10000/10000 [==============================] - 0s 50us/step
[0.8731090294837952, -0.016179607404768467, 1.762397668838501, 0.0, 0.3715]
10000/10000 [==============================] - 1s 59us/step
[0.8719294109344482, -0.020654971302300693, 1.7645137935638429, 0.0, 0.3595]
10000/10000 [==============================] - 1s 56us/step
[0.8771961116790772, 0.0004820874644443393, 1.7539101341247558, 0.0, 0.3704]
10000/10000 [==============================] - 1s 53us/step
[0.8737041703224182, -0.012114058296382428, 1.7595224029541017, 0.0, 0.3651]
10000/10000 [==============================] - 1s 55us/step
[0.8712271175384522, -0.0022786013967357574, 1.7447328397750854, 0.0, 0.3728]
10000/10000 [==============================] - 1s 54us/step
[0.8791037318229675, 0.015874538972228767, 1.7423329299926757, 0.0, 0.3768]
10000/10000 [==============================] - 1s 54us/step
[0.8706862325668335, -0.0018409744903445243, 1.743213441467285, 0.0, 0.3415]
10000/10000 [=============================

10000/10000 [==============================] - 1s 55us/step
[0.6203196290016174, 0.004544651648402214, 1.2360946062088012, 0.0, 0.6681]
10000/10000 [==============================] - 1s 50us/step
[0.611190832901001, -0.020121484452486037, 1.2425031517028808, 0.0, 0.6533]
10000/10000 [==============================] - 1s 55us/step
[0.6441631107330322, 0.04185585605949164, 1.2464703666687011, 0.0, 0.6546]
10000/10000 [==============================] - 1s 54us/step
[0.6504481995582581, 0.0034814962428063156, 1.297414902496338, 0.0, 0.611]
10000/10000 [==============================] - 1s 56us/step
[0.6112362073898315, -0.016045028972625733, 1.2385174409866333, 0.0, 0.6654]
10000/10000 [==============================] - 1s 53us/step
[0.6317313121795655, 0.03284350828528404, 1.2306191144943237, 0.0, 0.6912]
10000/10000 [==============================] - 1s 54us/step
[0.6187536979675293, -0.0025943635475821794, 1.2401017576217652, 0.0, 0.6507]
10000/10000 [==============================] - 1

10000/10000 [==============================] - 1s 55us/step
[14.209712370300293, 26.067600134277345, 2.351824598312378, 0.0, 0.098]
10000/10000 [==============================] - 1s 54us/step
[13.37063542327881, 24.364450158691405, 2.376820669174194, 0.0, 0.1135]
10000/10000 [==============================] - 1s 53us/step
[12.267789981079101, 22.114933251953126, 2.420646695327759, 0.0, 0.0892]
10000/10000 [==============================] - 1s 51us/step
[9.102778619384766, 15.790830072021484, 2.414727174758911, 0.0, 0.1028]
10000/10000 [==============================] - 1s 53us/step
[4.186281909942627, 5.966947753143311, 2.4056160469055174, 0.0, 0.1135]
10000/10000 [==============================] - 1s 56us/step
[1.6706795528411866, 0.9403078709602356, 2.4010512355804443, 1.0, 0.0974]
10000/10000 [==============================] - 1s 55us/step
[1.3764957748413087, 0.4505564215183258, 2.3024351322174073, 0.633, 0.1965]
10000/10000 [==============================] - 1s 55us/step
[1.394872

10000/10000 [==============================] - 1s 55us/step
[0.6162960761070252, 0.0020470367873087524, 1.230545116043091, 0.0, 0.6802]
10000/10000 [==============================] - 1s 56us/step
[0.6302319295883179, 0.03197123099416494, 1.2284926267623901, 0.0, 0.6707]
10000/10000 [==============================] - 1s 55us/step
[0.6208618525505066, -0.007942432315275073, 1.2496661367416382, 0.0, 0.6757]
10000/10000 [==============================] - 1s 55us/step
[0.603865029335022, -0.040335889983177185, 1.2480659450531006, 0.0, 0.6682]
10000/10000 [==============================] - 1s 55us/step
[0.6491188742637635, 0.06290317951440812, 1.2353345691680908, 0.0, 0.6435]
10000/10000 [==============================] - 1s 54us/step
[0.6252757669448853, -0.0008564422968775034, 1.251407976913452, 0.0, 0.63]
10000/10000 [==============================] - 0s 47us/step
[0.6209393746376037, 0.00919427226204425, 1.2326844758987427, 0.0, 0.689]
10000/10000 [==============================] - 0s 49

In [ ]:
cnn_ss_histories = {}
for num_examples in [100, 300, 500, 1000]:
    print(num_examples)
    cnn_ss_histories[num_examples] = train_test_gan(
        cnn_wgan_factory, x_train, y_train, x_test, y_test, epochs=50000, num_labels=num_examples, verbose=False
    )

100


/home/qkt/.local/share/virtualenvs/sgan-techniques-WxACZbr4/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/qkt/.local/share/virtualenvs/sgan-techniques-WxACZbr4/lib/python3.5/site-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


10000/10000 [==============================] - 3s 345us/step
[1.199020803833008, 0.0001444129419978708, 2.3978971458435057, 0.0, 0.0892]
10000/10000 [==============================] - 3s 295us/step
[1.2034447160720825, -0.00455792110785842, 2.4114473541259764, 0.0, 0.0868]
10000/10000 [==============================] - 3s 298us/step
[1.2173508432388305, 0.042682951325178145, 2.3920187324523927, 0.0, 0.098]
10000/10000 [==============================] - 3s 298us/step
[1.1863086349487304, -0.010958937323093415, 2.383576203918457, 0.0, 0.1822]
10000/10000 [==============================] - 3s 297us/step
[1.1819973251342772, -0.020649126720428467, 2.3846437767028807, 0.0, 0.1749]
10000/10000 [==============================] - 3s 296us/step
[1.1821890487670899, -0.02132643413543701, 2.385704529953003, 0.0, 0.157]
10000/10000 [==============================] - 3s 299us/step
[1.1808946201324464, -0.023594550281763077, 2.3853837936401368, 0.0, 0.098]
10000/10000 [==============================

In [ ]:
def dense_wgan_factory_test():
    from keras.optimizers import Adam
    
    dense_d = sgt.discriminators.dense((28, 28, 1), 10, unsupervised_activation='linear')
    dense_g = sgt.generators.dense((100,), (28, 28, 1))
    return sgt.gan.SGANBuilder(dense_g, dense_d, (28, 28, 1), (100,)) \
        .set_discriminator_loss([sgt.losses.wasserstein_loss, 'categorical_crossentropy']) \
        .set_generator_loss(sgt.losses.wasserstein_loss) \
        .set_discriminator_optimizer(Adam()) \
        .set_generator_optimizer(Adam()) \
        .build()

In [ ]:
dense_histories_adam = train_test_gan(dense_wgan_factory_test, x_train, y_train, x_test, y_test, epochs=12500, verbose=False)

In [ ]:
clip_value = 0.1

In [ ]:
dense_histories_large_clip = train_test_gan(dense_wgan_factory, x_train, y_train, x_test, y_test, epochs=12500, verbose=False)